# Polynomial Regression and Overfitting

In this module, you will learn how to work with polynomial regression in scikit-learn. We will also introduce the concept of overfitting and learn how to avoid this.

<b>Functions and attributes in this lecture: </b>
- `pandas:` - Pandas package with alias `pd`
 - `.copy()` - Make a copy of a pandas Dataframe
- `sklearn.preprocessing` - Submodule for preprocessing tools like PolynomialFeatures
  - `PolynomialFeatures` - For creating of polynomial features
  - `.fit_transform()` - Fitting and transforming the data to form new polynomial features
- `sklearn.metrics` - Submodule for metrics used to evaluate models
  - `mean_absolute_error` - Taking the mean-abosolute-error of a vector

In [471]:
# Non-sklearn packages
import numpy as np
import pandas as pd

# Sklearn packages
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [472]:
# Load in the diabetes dataset
X, y = load_diabetes(return_X_y=True, as_frame=True)

# Get the description of the dataset
print(load_diabetes()["DESCR"])

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

:Number of Instances: 442

:Number of Attributes: First 10 columns are numeric predictive values

:Target: Column 11 is a quantitative measure of disease progression one year after baseline

:Attribute Information:
    - age     age in years
    - sex
    - bmi     body mass index
    - bp      average blood pressure
    - s1      tc, total serum cholesterol
    - s2      ldl, low-density lipoproteins
    - s3      hdl, high-density lipoproteins
    - s4      tch, total cholesterol / HDL
    - s5      ltg, possibly log of serum triglycerides level
    - s6      glu, blood sugar level

Note: Each of these 10 feature variables have bee

## Manually Create Polynomial Features

We begin by manually creating features in Pandas and measure how this affects the error rate of our models.

In [473]:
# Check the correlation with the target variable
pd.concat([X,y], axis=1).corr()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
age,1.000000,0.173737,0.185085,0.335428,0.260061,0.219243,-0.075181,0.203841,0.270774,0.301731,0.187889
sex,0.173737,1.000000,0.088161,0.241010,0.035277,0.142637,-0.379090,0.332115,0.149916,0.208133,0.043062
bmi,0.185085,0.088161,1.000000,0.395411,0.249777,0.261170,-0.366811,0.413807,0.446157,0.388680,0.586450
bp,0.335428,0.241010,0.395411,1.000000,0.242464,0.185548,-0.178762,0.257650,0.393480,0.390430,0.441482
s1,0.260061,0.035277,0.249777,0.242464,1.000000,0.896663,0.051519,0.542207,0.515503,0.325717,0.212022
s2,0.219243,0.142637,0.261170,0.185548,0.896663,1.000000,-0.196455,0.659817,0.318357,0.290600,0.174054
s3,-0.075181,-0.379090,-0.366811,-0.178762,0.051519,-0.196455,1.000000,-0.738493,-0.398577,-0.273697,-0.394789
s4,0.203841,0.332115,0.413807,0.257650,0.542207,0.659817,-0.738493,1.000000,0.617859,0.417212,0.430453
s5,0.270774,0.149916,0.446157,0.393480,0.515503,0.318357,-0.398577,0.617859,1.000000,0.464669,0.565883
s6,0.301731,0.208133,0.388680,0.390430,0.325717,0.290600,-0.273697,0.417212,0.464669,1.000000,0.382483


In [474]:
# Selecting only some of the columns
X_selected = X[["bmi","bp","s4","s5"]]

In [475]:
# Creating a copy for further modification
X_cross_terms = X_selected.copy()

In [476]:
# Manually creating cross_terms
X_cross_terms["bmi_times_bp"] = X_cross_terms["bmi"] * X_cross_terms["bp"]
X_cross_terms["bp_times_s5"] = X_cross_terms["bp"] * X_cross_terms["s5"]

In [477]:
# Shows the new columns we have created
X_cross_terms.head()

,bmi,bp,s4,s5,bmi_times_bp,bp_times_s5
0,0.061696,0.021872,-0.002592,0.019907,0.001349,0.000435
1,-0.051474,-0.026328,-0.039493,-0.068332,0.001355,0.001799
2,0.044451,-0.005670,-0.002592,0.002861,-0.000252,-0.000016
3,-0.011595,-0.036656,0.034309,0.022688,0.000425,-0.000832
4,-0.036385,0.021872,-0.002592,-0.031988,-0.000796,-0.000700


In [478]:
# Showing the correlation again
pd.concat([X_cross_terms,y], axis=1).corr()

,bmi,bp,s4,s5,bmi_times_bp,bp_times_s5,target
bmi,1.000000,0.395411,0.413807,0.446157,0.059820,-0.002844,0.586450
bp,0.395411,1.000000,0.257650,0.393480,0.117997,0.099798,0.441482
s4,0.413807,0.257650,1.000000,0.617859,-0.087451,0.026749,0.430453
s5,0.446157,0.393480,0.617859,1.000000,-0.002511,0.083321,0.565883
bmi_times_bp,0.059820,0.117997,-0.087451,-0.002511,1.000000,0.410844,0.147114
bp_times_s5,-0.002844,0.099798,0.026749,0.083321,0.410844,1.000000,0.099437
target,0.586450,0.441482,0.430453,0.565883,0.147114,0.099437,1.000000


In [479]:
# Splitting into test sets and training sets
X_train, X_test, y_train, y_test = train_test_split(X_cross_terms, y, test_size=0.33, random_state=42)

In [480]:
# Creating, training, and predicting the polynomial model
poly_reg = LinearRegression()
poly_reg.fit(X_train,y_train)
y_poly_pred = poly_reg.predict(X_test)

In [481]:
# Use mean-absolute-error to measure the model
from sklearn.metrics import mean_absolute_error 
print(f"Poly Error: {mean_absolute_error(y_poly_pred, y_test)}")

Poly Error: 42.93444838156814


In [482]:
# Create a linear model
lin_reg = LinearRegression()
lin_reg.fit(X_train[["bmi","bp","s4","s5"]],y_train)
y_lin_pred = lin_reg.predict(X_test[["bmi","bp","s4","s5"]])

In [483]:
# Shows score of the purely linear model
print(f"Lin Error: {mean_absolute_error(y_lin_pred, y_test)}")

Lin Error: 43.56261335601803


## Using Scikit-Learns Polynomial Features

We now introduce scikit-learn's built in PolynomialFeatures for handling polynomial featues.

In [484]:
# Importing polynomial features
from sklearn.preprocessing import PolynomialFeatures

In [485]:
# We are working with four features
X_selected.head()

,bmi,bp,s4,s5
0,0.061696,0.021872,-0.002592,0.019907
1,-0.051474,-0.026328,-0.039493,-0.068332
2,0.044451,-0.005670,-0.002592,0.002861
3,-0.011595,-0.036656,0.034309,0.022688
4,-0.036385,0.021872,-0.002592,-0.031988


In [486]:
# Creates polynomial features automatically
poly_features = PolynomialFeatures(degree=2,interaction_only=True,include_bias=False)
X_poly = poly_features.fit_transform(X_selected)

In [487]:
# Split into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.33, random_state=42)

In [488]:
# Create model, train, and test
poly_reg = LinearRegression()
poly_reg.fit(X_train,y_train)
y_pred = poly_reg.predict(X_test)

In [489]:
# Getting the mean-absolute-error score
print(f"Poly Error: {mean_absolute_error(y_pred, y_test)}")

Poly Error: 43.19184830684269


## Fitting Everything Into a Pipeline

It is time to fit the previous steps into a pipeline for better reproducibility and to avoid data leakage.

In [490]:
# We are working with four features
X_selected.head()

,bmi,bp,s4,s5
0,0.061696,0.021872,-0.002592,0.019907
1,-0.051474,-0.026328,-0.039493,-0.068332
2,0.044451,-0.005670,-0.002592,0.002861
3,-0.011595,-0.036656,0.034309,0.022688
4,-0.036385,0.021872,-0.002592,-0.031988


In [491]:
# Dividing into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.33, random_state=42)

In [503]:
# Creating a pipeline (use scaler after the polynomial features - small number convergence)
pipe = Pipeline([
        ("poly_features",PolynomialFeatures(degree=2)),
        ("scaler", StandardScaler()),
        ("poly_reg", LinearRegression() 
    )],
)

In [504]:
# Fitting and predicting
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

In [505]:
# We get the same score
print(f"Poly Error: {mean_absolute_error(y_pred, y_test)}")

Poly Error: 45.1044072020844


## Checking if you are Overfitting

We will now loop through several polynomial degrees to check which one is best.

In [495]:
# We are working with four features
X_selected.head()

,bmi,bp,s4,s5
0,0.061696,0.021872,-0.002592,0.019907
1,-0.051474,-0.026328,-0.039493,-0.068332
2,0.044451,-0.005670,-0.002592,0.002861
3,-0.011595,-0.036656,0.034309,0.022688
4,-0.036385,0.021872,-0.002592,-0.031988


In [506]:
# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.33, random_state=42)

In [519]:
# Checking the error for various degrees
for i in range(1,6):
    pipe = Pipeline([
            ("poly_features",PolynomialFeatures(degree=i)),
            ("scaler", StandardScaler()),
            ("poly_reg", LinearRegression()) 
    ])
    pipe.fit(X_train, y_train)
    y_pred_train = pipe.predict(X_train)
    y_pred_test = pipe.predict(X_test)
    train_diff = round(mean_absolute_error(y_pred_train,y_train),2)
    test_diff = round(mean_absolute_error(y_pred_test,y_test),2)
    print(f"| Degree {i} on train: {train_diff} | Degree {i} on test: {test_diff} |")

| Degree 1 on train: 47.11 | Degree 1 on test: 43.56 |
| Degree 2 on train: 46.03 | Degree 2 on test: 43.92 |
| Degree 3 on train: 44.62 | Degree 3 on test: 44.59 |
| Degree 4 on train: 40.35 | Degree 4 on test: 48.52 |
| Degree 5 on train: 33.74 | Degree 5 on test: 81.03 |
